# Accessibility to public transport

In [ ]:
# If using colab
# Takes around 2-3 min
# !pip install "UrbanAccessAnalyzer[osm,plot,h3] @ git+https://github.com/CityScope/UrbanAccessAnalyzer.git"
# !pip install matplotlib mapclassify folium
# !apt-get install -y osmium-tool


# Restart notebook after installing this if needed

In [ ]:
import sys 
sys.path.append('/home/miguel/Documents/Proyectos/PTLevelofService/gtfs/pyGTFSHandler')
sys.path.append('/home/miguel/Documents/Proyectos/PTLevelofService/accessibility/UrbanAccessAnalyzer')

In [ ]:
import os
from datetime import datetime, date, timedelta, time
import pandas as pd
import polars as pl
import geopandas as gpd
import os

import osmnx as ox

import matplotlib.pyplot as plt

import UrbanAccessAnalyzer.isochrones as isochrones
import UrbanAccessAnalyzer.graph_processing as graph_processing
import UrbanAccessAnalyzer.osm as osm
import UrbanAccessAnalyzer.utils as utils
import UrbanAccessAnalyzer.h3_utils as h3_utils
import UrbanAccessAnalyzer.population as population

from pyGTFSHandler.feed import Feed
from pyGTFSHandler.downloaders.mobility_database import MobilityDatabaseClient
import pyGTFSHandler.plot_helper as plot_helper
import pyGTFSHandler.gtfs_checker as gtfs_checker
import pyGTFSHandler.processing_helper as processing_helper

import zipfile
import numpy as np

#### Results folder

In [ ]:
results_path = os.path.normpath("output")
os.makedirs(results_path,exist_ok=True)

## 1 Area of interest

Area of interest (aoi): Polygon to do the analisys

Option 1: Write the city name

In [ ]:
city_name = "Parla, España"
city_filename = utils.sanitize_filename(city_name)
city_results_path = os.path.join(results_path,city_filename)
os.makedirs(city_results_path,exist_ok=True)
aoi = utils.get_city_geometry(city_name)
geo_suggestions = utils.get_geographic_suggestions_from_string(city_name,user_agent="app")
geo_suggestions

Option 2: Load your own file (.gpkg or .shp)

In [ ]:
# aoi = gpd.read_file("")
# city_name = ""

Use UTM coords and creat aoi_download with a buffer of X meters

In [ ]:
aoi = gpd.GeoDataFrame(geometry=[aoi.union_all()],crs=aoi.crs) # Ensure there is only one polygon
aoi = aoi.to_crs(aoi.estimate_utm_crs()) # Convert to utm

aoi_download = aoi.buffer(2000) # Area to do streets and poi requests 
# It should be the maximum buffer we will use but there is the risk of downloading an area that is too large

Map of your area of interest and the download area (aoi_buffer)

In [ ]:
m=aoi_download.explore(color='green')
m=aoi.explore(m=m,color='red')
m

## 2 Public transport data

### 2.1 Download GTFS feeds worldwide

This example is for the MobilityData API

This is the organization responsible for the GTFS standard and has info from almost all the world

In [ ]:
# Request your refresh token here: https://mobilitydatabase.org/ 
refresh_token = 'AMf-vByYiwMAni1pw6yTpwgwwYFc8HR4y0zUKZGPT4sjJ0wUrIXOfVxF1KotRIvEgAseaaNheL8YczJiCILb6o2PUh-8zjA-qQURzEc8tELlwFiDopMoqJnkDf13AqNaGGnnzTDmYM20AWEquUxcYFAB8Q3e5rI2DcTBSQuiUdHL8bi48xmUJk3tayHpnoicoppi_evDcWYODwOJFcwnta3K7f718w7R2JRM0zDEOYw7nI7thrQa9462BENdpv8zv8mEbBssEa189k6YcV__sQAZlng2EcsCGA'
api = MobilityDatabaseClient(refresh_token)

Find Feeds on the API

In [ ]:
feeds = api.search_gtfs_feeds(
    country_code=geo_suggestions['country_codes'],
    subdivision_name=geo_suggestions['subdivision_names'], # This info is not always in the feeds metadata. Comment this if you did not find all feeds.
    municipality=geo_suggestions['municipalities'], # This info is not always in the feeds metadata. Comment this if you did not find all feeds.
    is_official=None, # Set to True if you only want official feeds
    #aoi=aoi, # You could comment the rest of search args and use only aoi but for now the API seems to not do this very well as the metadata is often wrong.
)

for f in feeds:
    print(f['provider'])

Download current active files

In [ ]:
gtfs_path = os.path.join(results_path,"gtfs_files") 
os.makedirs(gtfs_path,exist_ok=True)

In [ ]:
file_paths = api.download_feeds(
    feeds=feeds,
    download_folder=gtfs_path,
    overwrite=False
)

### Optional: Properly check all gtfs files for validity

This takes a few minutes. If you skip the code will usually ignore wrong gtfs file rows without logging or solving it.

In [ ]:
# # Check and fix the gtfs files (This takes a few minutes). Set check_files = False in Feed to load faster

# new_gtfs_path = os.path.join(results_path,"revised_gtfs_files") 
# os.makedirs(new_gtfs_path)

# new_file_paths = []
# for f in file_paths:
#     filename = os.path.splitext(os.path.basename(f))[0]
#     if os.path.isdir(os.path.join(new_gtfs_path,filename)):
#         new_file_paths.append(os.path.join(new_gtfs_path,filename))
#     else:
#         new_file_paths.append(gtfs_checker.preprocess_gtfs(f,new_gtfs_path))

# file_paths = new_file_paths

### 2.2 GTFS process

#### 2.2.1 Create the gtfs object

This will do:

- Load all .txt files of all gtfs folders given.
- Select only the stops from stops.txt inside the area of interest.
- Crop all trips in stop_times.txt with the stops inside the aoi + 1 more stop.
- Check the stop_sequence in stop_times.txt.
- Deal correctly with trips starting on one day and ending in the following day: hours always in 0-24 range but those trips are marked as next_day True. New service_ids are created to deal with that.
- If the file has frequencies.txt this is processed too dealing with the next day problem.
- If departure or arrival times are empty they get filled.
- A shape direction col is computed as the mean heading of the vector between stop coordinates to mean of the remainning stops coordinates.
- GTFS shapes are for now computed from the stop coordinates.

In [ ]:
gtfs = Feed(
    file_paths,
    aoi=aoi,
    stop_group_distance=100, # Group stops into one that are less than x meters apart. This creates or updates the parent_station column
    start_date=datetime(day=1,month=11,year=2025),
    end_date=datetime(day=1,month=1,year=2026),
)

#### 2.2.2 Service Intensity

This is the number of vehicles that arrive at each stop every day multiplied by the number of stops:

$\text{Service Intensity} = (\text{Number of vehicles per stop}) \times (\text{Number of stops})$

In [ ]:
service_intensity = gtfs.get_service_intensity_in_date_range(
    start_date=None, # If None take the feed min date
    end_date=None, # If None take the feed max date
    date_type=None, # Could be something like 'holiday', 'weekday', or 'monday' to only consider some dates from the range.
    by_feed=True
)
service_intensity = service_intensity.to_pandas()
plot_helper.plot_service_intensity(service_intensity)

Select the most representative business day in a date range

In [ ]:
selected_service_intensity = gtfs.get_service_intensity_in_date_range(
    start_date=None, # If None take the feed min date
    end_date=None, # If None take the feed max date
    date_type='businessday' # Could be something like 'holiday', 'businessday', 'non_businessday', or 'monday' to only consider some dates from the range.
)
selected_service_intensity = selected_service_intensity.to_pandas()
idx = processing_helper.most_frequent_row_index(selected_service_intensity['service_intensity'])
selected_day = selected_service_intensity.iloc[idx]['date'].to_pydatetime()
selected_day

#### 2.2.3 Parameters, dates and times

In [ ]:
# date and time
date = selected_day # date in this format date(year=2025,month=4,day=1)
start_time = time(hour=8)
end_time = time(hour=20)

route_types = 'all' # Valid values are 'tram' 'subway' 'rail' 'bus' 'ferry' 'cable_car' 'gondola' 'funicular' 
# or any list combining those like ['rail', 'subway']

stop_id = "parent_station" # Use the stop groups created with arg stop_group_distance in Feed to group neraby stops into one
# You could choose 'stop_id' too

#### 2.2.4 Service quality

Service quality is evaluated depending the route type and the mean frequency in the selected time interval.

By default processing_helper.SERVICE_MATRIX to give grades

In [ ]:
# Lets see what grade is given by default depending on route type and frequency.

processing_helper.SERVICE_MATRIX # interval in minutes. The grading for stops is 1 for best - 12 for worst.

In [ ]:
simplified_route_type_mapping = {
    'bus':'all',
    'tram':[0,3,4,5,6,7],
    'rail':[1,2]
}

# 0 - tram 
# 1 - subway 
# 2 - rail 
# 3 - bus
# 4 - ferry 
# 5 - cable car 
# 6 - gondola 
# 7 - funicular


#### 2.2.4 Average waiting time (interval) at stops

Interval is in *minutes*

In this example we use the *shape_direction* mode 

- Uses direction (shape_direction) in degrees from north that every 'trip_id' is pointing to computed at every stop and every trip
- Creates 'n_divisions' * 2 groups (*2 to get outbound and inbound directions independently) by clustering the trip shape directions 
- If how = 'best' means the interval is computed only for the best of all divisions at every stop 

In [ ]:
stop_interval_df = []
for simplified_route_type in simplified_route_type_mapping.keys():
    route_types = simplified_route_type_mapping[simplified_route_type]
    stop_interval_df.append(
        gtfs.get_mean_interval_at_stops(
            date=selected_day,
            start_time=start_time,
            end_time=end_time,
            route_types=route_types, 
            by = "shape_direction", # Interval is computed for all 'trip_id' grouped by this column and sorted by 'departure_time'
            at = stop_id, # Where to compute the interval 'stop_id' 'parent_station'
            how = "best", 
            # 'best' pick the route with best interval, 
            # 'mean' Combine all intervals of all routes, 
            # 'all' return results per stop and route
            n_divisions=1, # Number of divisions for by = 'shape_direction'
        ).with_columns(pl.lit(simplified_route_type).alias("simplified_route_type"))
    )

stop_interval_df = (
    pl.concat(stop_interval_df)
).to_pandas()
stop_interval_df = gtfs.add_stop_coords(stop_interval_df)
stop_interval_df = gtfs.add_route_names(stop_interval_df)
stop_interval_df = gpd.GeoDataFrame(
    stop_interval_df,
    geometry=gpd.points_from_xy(stop_interval_df['stop_lon'],y=stop_interval_df['stop_lat']),
    crs=4326
)
stop_interval_df = stop_interval_df[stop_interval_df.geometry.is_valid]
stop_interval_df["service_quality"] = stop_interval_df.apply(
    lambda row: processing_helper.assign_service_quality_to_interval(
        row["mean_interval"],
        row["simplified_route_type"],
        service_matrix=processing_helper.SERVICE_MATRIX
    ),
    axis=1
)
stop_interval_df = stop_interval_df.sort_values("service_quality").drop_duplicates(stop_id,keep="first")
stop_interval_df = stop_interval_df.sort_values(stop_id).reset_index(drop=True)
stop_interval_df.to_file(os.path.join(city_results_path,"stops.gpkg"))
stop_interval_df

In [ ]:
m = stop_interval_df[
    [
        stop_id,
        "mean_interval",
        "service_quality",
        "stop_name",
        "route_names",
        "route_type_texts",
        "simplified_route_type",
        "geometry",
    ]
].explore(
    column=f"service_quality",
    cmap="Blues_r",
    vmin=1,
    vmax=8,
    style_kwds={
        "color": "black",  # Border color
        "weight": 1,  # Border thickness
        "opacity": 1.0,  # Border opacity
        "fillOpacity": 1,
        "radius": 6,
    },
)
m.save(city_results_path + "/stops.html")
m

## 3 Street graph

### 3.1 Regionwise file and cropping

- Download best regionwise pbf file. (Covers a large area)

- Crop it to cover our area of interest and save it in .osm format

In [ ]:
osm_xml_file = os.path.normpath(city_results_path+f"/streets.osm")
streets_graph_path = os.path.normpath(city_results_path+f"/streets.graphml")
streets_path = os.path.normpath(city_results_path+f"/streets.gpkg")
level_of_service_streets_path = os.path.normpath(city_results_path+f"/level_of_service_streets.gpkg")
population_results_path = os.path.normpath(city_results_path+f"/population.gpkg")

In [ ]:
# Select what type of street network you want to load
network_filter = osm.osmium_network_filter("walk+bike+primary")
# Download the region pbf file crop it by aoi and convert to osm format
osm.geofabrik_to_osm(
    osm_xml_file,
    input_file=results_path,
    aoi=aoi_download,
    osmium_filter_args=network_filter,
    overwrite=False
)

### 3.2 Load to osmnx

This way the street network is a networkx graph

In [ ]:
# Load
G = ox.graph_from_xml(osm_xml_file)
# Project geometry coordinates to UTM system to allow euclidean meassurements in meters (sorry americans)
G = ox.project_graph(G,to_crs=aoi.estimate_utm_crs())
# Save the graph in graphml format to avoid the slow loading process
ox.save_graphml(G,streets_graph_path)

### 3.3 Simplify graph

Edges with length smaler than X meters are deleted and its nodes merged

In [ ]:
min_edge_length = 30

G = graph_processing.simplify_graph(G,min_edge_length=min_edge_length,min_edge_separation=min_edge_length*2,undirected=True)
# Save the result in graphml format
ox.save_graphml(G,streets_graph_path)

street_edges = ox.graph_to_gdfs(G,nodes=False)
street_edges = street_edges.to_crs(aoi.crs)
street_edges.to_file(streets_path)

### 3.4 Add Points of interest to graph

In [ ]:
G, osmids = graph_processing.add_points_to_graph(
    stop_interval_df,
    G,
    max_dist=100+min_edge_length, # Maximum distance from point to graph edge to project the point
    min_edge_length=min_edge_length # Minimum edge length after adding the new nodes
)
stop_interval_df['osmid'] = osmids # Add the ids of the nodes in the graph to points

## 4 Compute isochrones

### 4.1 Distance matrix

We need a DISTANCE_MATRIX to relate level of service classes to service qualities and distance to the service.

and we need a LEVEL_OF_SERIVCES list to order the level of services classes form best to worst (or leave it up to the code is the matrix is very symetric).

In [ ]:
distance_matrix = processing_helper.DISTANCE_MATRIX
distance_matrix

In [ ]:
level_of_services = processing_helper.LEVEL_OF_SERVICES
level_of_services

### 4.2 Isochrones

In [ ]:
level_of_service_graph = isochrones.graph(
    G,
    stop_interval_df,
    distance_matrix,
    service_quality_col = None, # If all points have the same quality this could be None
    level_of_services = level_of_services, # could be None and it will set to the sorted unique values of the matrix
    min_edge_length = min_edge_length # Do not add new nodes if there will be an edge with less than this length
)
# Save edges as gpkg
level_of_service_nodes, level_of_service_edges = ox.graph_to_gdfs(level_of_service_graph)
level_of_service_edges.to_file(level_of_service_streets_path)

#### Lets visualize the results on a map

### 4.3 Convert to H3

In [ ]:
h3_resolution = 10

In [ ]:
ls_h3_df = h3_utils.from_gdf(
    level_of_service_edges,
    resolution=h3_resolution,
    value_column='level_of_service',
    value_order=level_of_services,
    buffer=10
)

ls_h3_df

In [ ]:
m = None

# m = street_edges.explore(
#     m = m,
#     color='black'
# )

m = h3_utils.h3_to_gdf(ls_h3_df).explore(
    m=m,
    column='level_of_service',
    cmap="RdYlGn_r",
    style_kwds={
        "color": None,   # no border color
        "weight": 0,     # border width
        "fillOpacity": 0.3,  # optional fill transparency
    }
)

m = level_of_service_edges.explore(
    m=m,
    column='level_of_service',
    cmap="RdYlGn_r",
    style_kwds={
        "weight": 3,        # thickness of the polygon/line border
    }
)

m = stop_interval_df[
    [
        stop_id,
        "mean_interval",
        "service_quality",
        "stop_name",
        "route_names",
        "route_type_texts",
        "simplified_route_type",
        "geometry",
    ]
].explore(
    m=m,
    column=f"service_quality",
    cmap="Blues_r",
    vmin=1,
    vmax=8,
    style_kwds={
        "color": "black",  # Border color
        "weight": 1,  # Border thickness
        "opacity": 1.0,  # Border opacity
        "fillOpacity": 1,
        "radius": 6,
    },
)
m.save(city_results_path + "/level_of_service.html")
m

## 5 Population

### 5.1 Download Worldpop tif file

- One file for every country
- 100m pixel size
- tif format
- available from 2000 to 2030
- gender and age

In [ ]:
population_file = population.download_worldpop_population(
    aoi_download,
    2025,
    folder=results_path,
    resolution="100m",
)

In [ ]:
pop_h3_df = h3_utils.from_raster(population_file,aoi=aoi_download,resolution=h3_resolution)

### 5.2 Assign level of service to each population cell

In [ ]:
results_h3_df = ls_h3_df.merge(pop_h3_df,on='h3_cell',how='outer')
results_h3_df = h3_utils.h3_to_gdf(results_h3_df).to_crs(aoi.crs)
results_h3_df = results_h3_df[results_h3_df.intersects(aoi.union_all())]
results_h3_df.to_file(population_results_path)
results_h3_df

In [ ]:
m = None

# m = street_edges.explore(
#     m = m,
#     color='black'
# )

m = level_of_service_edges.explore(
    m=m,
    column='level_of_service',
    cmap="RdYlGn_r",
)

m = stop_interval_df[[
    #"name",
    "geometry"
]].explore(
    m=m,
    color="green",
    style_kwds={
        "color": "black",       # Border color
        "weight": 1,            # Border thickness
        "opacity": 1.0,         # Border opacity
        "fillOpacity": 1,
        "radius": 4,
    },
)

pop_gdf_points = results_h3_df.copy()
pop_gdf_points.geometry = pop_gdf_points.geometry.centroid
pop_gdf_points = pop_gdf_points.dropna(subset=['population'])
pop_gdf_points = pop_gdf_points[pop_gdf_points['population'] > 1]
m=pop_gdf_points.explore(
    m=m,
    column="level_of_service",            # color by service level
    cmap="RdYlGn_r",
    legend=True,
    tooltip=["population", "level_of_service"],
    marker_type="circle_marker",
    style_kwds={
        # define a dynamic radius for each feature
        "style_function": lambda x: {
            "radius": (np.log1p(x["properties"]["population"]) /
                       np.log1p(pop_gdf_points["population"].max()) * 5),
            "color": "black",      # border color
            "weight": 1,           # border thickness
            "opacity": 1.0,        # border opacity
            "fillOpacity": 1.0,    # fill opacity
        },
    },
)
m.save(city_results_path + "/population.html")
import webbrowser 
webbrowser.open(city_results_path + "/population.html")
m

Important files:

- streets.gpkg Has the street geometry as lines (all streets)
- level_of_service_streets.gpkg Has the street geometry as lines with the level of service (only streets with level of service)
- population.gpkg Is a grid with population and level of service